In [1]:
from process import event_preprocess, argument_preprocess,load_event, load_argument, Schema2Label
from src.seqlabel_utils import pos2bio
from src.preprocess.str_utils import get_useless_chars
from process import text_preprocess, gen_pos
from src.dataset.converter import single_text, double_text
from evaluation import extract_multilabel
import pandas as pd 
import numpy as np 
import ast 
import re 
pd.set_option('display.max_colwidth', None)

In [2]:
train_e = load_event('./trainsample/duee_train.json')
valid_e = load_event('./trainsample/duee_dev.json')
test_e = load_event('./trainsample/duee_test2.json')
schema = Schema2Label('./trainsample/duee_event_schema.json')
train_e.shape, valid_e.shape, test_e.shape

((11908, 3), (1492, 3), (34904, 2))

In [3]:
useless_chars = get_useless_chars(train_e['text'].values.tolist() +\
                                  valid_e['text'].values.tolist() +\
                                  test_e['text'].values.tolist())

## pipeline

### 事件抽取方案1：Trigger Sequence Labelling

In [4]:
train_e = event_preprocess(train_e, useless_chars)
valid_e = event_preprocess(valid_e, useless_chars)
test_e = event_preprocess(test_e, useless_chars)
valid_e.to_csv('./trainsample/valid_event.csv')
train_e.to_csv('./trainsample/train_event.csv')
test_e.to_csv('./trainsample/test_event.csv')

0 out of 11908 not match
0 out of 1492 not match


from src.dataset.converter import single_text
single_text(train_e['id'].values, train_e['clean_text'].values, train_e['event_bio_label'].values,
            './trainsample','train_event_bio')
single_text(valid_e['id'].values, valid_e['clean_text'].values, valid_e['event_bio_label'].values, 
            './trainsample','valid_event_bio')
single_text(test_e['id'].values, test_e['clean_text'].values, None ,'./trainsample','test_event_bio')

In [25]:
train_e['clean_text'][4][13:15]

'裁员'

In [26]:
train_a.head()

,id,text,event_type,trigger,arguments,clean_text,pos_map,event_text,arguments_adjust,arguments_pos,argument_bio_label
0,409389c96efe78d6af1c86e0450fd2d7,"雀巢裁员4000人:时代抛弃你时,连招呼都不会打!",组织关系-裁员,裁员,"[[裁员方, 雀巢, 0], [裁员人数, 4000人, 4]]","雀巢裁员4000人:时代抛弃你时,连招呼都不会打!","{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: ...","组织关系-裁员[SEP]雀巢裁员4000人:时代抛弃你时,连招呼都不会打!","[[裁员方, 雀巢, 12], [裁员人数, 4000人, 16]]","[[裁员方, 12, 13], [裁员人数, 16, 20]]","[O, O, O, O, O, O, O, O, O, O, O, O, B-裁员方, I-..."
1,5aec2b5b759c5f8f42f9c0156eb3c924,"美国“未来为”子公司大幅度裁员,这是为什么呢?任正非正式回应",组织关系-裁员,裁员,"[[裁员方, 美国“未来为”子公司, 0]]","美国“未来为”子公司大幅度裁员,这是为什么呢?任正非正式回应","{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: ...","组织关系-裁员[SEP]美国“未来为”子公司大幅度裁员,这是为什么呢?任正非正式回应","[[裁员方, 美国“未来为”子公司, 12]]","[[裁员方, 12, 21]]","[O, O, O, O, O, O, O, O, O, O, O, O, B-裁员方, I-..."
2,82c4db0b0b209565485a1776b6f1b580,这一全球巨头“凉凉” “捅刀”华为后 裁员5000 现市值缩水800亿,组织关系-裁员,裁员,"[[裁员人数, 5000, 21]]",这一全球巨头“凉凉”“捅刀”华为后裁员5000现市值缩水800亿,"{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: ...",组织关系-裁员[SEP]这一全球巨头“凉凉”“捅刀”华为后裁员5000现市值缩水800亿,"[[裁员人数, 5000, 31]]","[[裁员人数, 31, 34]]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,1f0eac3455f94c9d93dbacc92bbf4aec,被证实将再裁员1800人 AT&T在为落后的经营模式买单,组织关系-裁员,裁员,"[[裁员人数, 1800人, 7], [裁员方, AT&T, 13]]",被证实将再裁员1800人AT&T在为落后的经营模式买单,"{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: ...",组织关系-裁员[SEP]被证实将再裁员1800人AT&T在为落后的经营模式买单,"[[裁员人数, 1800人, 19], [裁员方, AT&T, 24]]","[[裁员人数, 19, 23], [裁员方, 24, 27]]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,d5b9a75b8d1dd37f07667ed72bf69c4f,"又一网约车巨头倒下:三个月裁员835名员工,滴滴又该何去何从",组织关系-裁员,裁员,"[[裁员人数, 835名员工, 15]]","又一网约车巨头倒下:三个月裁员835名员工,滴滴又该何去何从","{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: ...","组织关系-裁员[SEP]又一网约车巨头倒下:三个月裁员835名员工,滴滴又该何去何从","[[裁员人数, 835名员工, 27]]","[[裁员人数, 27, 32]]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
